In [1]:
pip install kafka-python

     -------------------------------------- 246.5/246.5 kB 1.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps
import json

In [2]:
producer = KafkaProducer(bootstrap_servers=['13.53.36.24:9092'],
                         value_serializer=lambda x: 
                         dumps(x, default=str).encode('utf-8'))

NoBrokersAvailable: NoBrokersAvailable

In [3]:
producer.send('demo_test', value={'hello':'world'})

NameError: name 'producer' is not defined

In [4]:
df = pd.read_excel("data/new_customers.xlsx")

In [5]:
df.head()

,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company
0,Andrew Mccall,37,9935.53,1,7.71,8,2011-08-29 18:37:00,"38612 Johnny Stravenue Nataliebury, WI 15717-8316",King Ltd
1,Michele Wright,23,7526.94,1,9.28,15,2013-07-22 18:19:00,"21083 Nicole Junction Suite 332, Youngport, ME...",Cannon-Benson
2,Jeremy Chang,65,100.00,1,1.00,15,2006-12-11 07:48:00,"085 Austin Views Lake Julialand, WY 63726-4298",Barron-Robertson
3,Megan Ferguson,32,6487.50,0,9.40,14,2016-10-28 05:32:00,"922 Wright Branch North Cynthialand, NC 64721",Sexton-Golden
4,Taylor Young,32,13147.71,1,10.00,8,2012-03-20 00:36:00,Unit 0789 Box 0734 DPO AP 39702,Wood LLC


In [6]:
import joblib
model = joblib.load('training/logistic_regression_model.pkl')

In [7]:
# Select features used for training the model
selected_features = ['Names', 'Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites']

In [8]:
while True:
    # Sample a row from the dataframe
    row = df.sample(1)

    # Extract features for prediction
    features = row[['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites']].values.tolist()[0]

    # Predict using the pre-trained model
    prediction = model.predict([features])[0]

    # Remove columns that start with certain words but keep the exact names
    columns_to_remove = [col for col in row.columns if col.startswith(('Names', 'Onboard_date', 'Company', 'Location')) and col not in ('Names', 'Onboard_date', 'Company', 'Location')]
    row = row.drop(columns=columns_to_remove)

    # Add the prediction to the dictionary
    dict_row = row.to_dict(orient='records')[0]
    dict_row['Prediction'] = prediction

    # Send the data row with the prediction to the Kafka topic
    producer.send('demo_test', value=dict_row)

    sleep(1)

NameError: name 'producer' is not defined

In [7]:
producer.flush() #clear data from kafka server